<a href="https://colab.research.google.com/github/jsstar522/hunkim_ML/blob/master/02_NeuralNet_Backpropagation/01_Backpropagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Backpropagation

## Neural Network에서 계산량 구하기

Neural Network는 수많은 Layer와 노드(perceptron)으로 구성됩니다. 784개의 데이터를 Input layer에 집어넣고 10개 노드를 가진 1st hidden layer, 10개의 노드를 가진 2nd hidden layer가 있을 때, 필요한 weight와 bias는 몇개이며 계산량은 얼만큼 될까요?

![이미지](1.jpg)
*[그림1]하나의 노드에는 Input data만큼의 w(Weight)와 하나의 bias가 필요하다*

**이렇게 Neural Network는 엄청나게 많은 양의 가중치(Weight)가 필요합니다.** 계산량도 어마어마해질 뿐만 아니라 단순한 Machine learning에서 도출된 Cost Function (Error)처럼 식을 구하는 것조차 힘듭니다. Hypothesis를 통해 나온 1차 측정값 (representation)을 다시 새로운 2차 Hypothesis에 넣어야 하므로 엄청나게 복잡한 식이 나오겠죠. 이러한 복잡한 식으로 Error를 구하기는 불가능하다는 것을 *Marvin Minsky* 박사가 1969년에 증명했습니다. 이런 복잡한 식으로 미분을 해서 최소값을 찾아가는 과정...!@$%# 말이 안되죠. 이렇게 인공지능의 침체기가 시작됐지만 Backpropagation이 등장하면서 다시 불이 붙기 시작하죠.


## Backpropagation

**Backpropagation은 중간에 계산과정이 어떻게 됐던지 간에 오차가 줄어드는 방향을 근거 있게 설명합니다. 예측값이 나오면 Error을 계산하고 오류에 영향을 가장 크게 미치는 Weight값을 찾습니다. 그리고 가중치(Weight)를 Update 하죠.** 

수많은 데이터가 수많은 다음 layer의 노드로 이동하는 과정 중에 하나만 확대해서 그려보겠습니다.

![이미지](2.jpg)
*[그림2]하나의 input data가 하나의 노드로 이동하는 과정*

이렇게 예측값이 나왔을 때 w(weight)가 예측값(f)에 미치는 영향이 얼마나 되는지 계산해보죠. **얼마나 영향을 미치는지 알아보려면 w값이 변할 때, f값은 얼마나 변하는지 구하면 됩니다. 즉, f를 w에 대해 편미분하면 되죠.**

$$\frac{af}{aw}$$

f = g + b이므로,

$$\frac{af}{ag}\frac{ag}{aw}$$

로 표현되어야 합니다. (이걸 미적분학에서 **Chain Rule**이라고 합니다.)

위 그림에서,
$$\frac{af}{ag} = 1,\frac{ag}{aw} = x_{(x=5)}$$

이므로

$$\frac{af}{aw} = 5$$

가 됩니다.

이 의미는 w가 1증가 할 때, f가 5배만큼 증가한다는 뜻이죠. w가 큰 영향을 미치는군요.

**같은 방식으로 f를 Error로 바꿔서 풀면 Error에 큰 영향을 미치는 w(weight)를 찾을 수 있게 됩니다. 그렇게 w를 "Optimizing" 해나가는 것입니다.**

![이미지](3.jpg)
*[그림3] 거꾸로 편미분해 나아가면 최초 w(weight) 값이 최종 f값에 얼마나 영향을 미치는지 알 수 있다.*

[그림3]을 보면 왜 Backpropagation이라는 이름이 붙었는지 느낌이 올겁니다. 

**이렇게 수많은 Layer를 거친 Hypothesis를 구해 복잡하게 미분을 하지 않고, 각 Layer에서의 편미분 값을 이용해 가중치(weight)의 영향력을 알 수 있습니다. 이 방법으로 Error에 미치는 영향력도 알 수 있고 Optimizing 하는 데 더 수월해졌습니다.**

## XOR 문제 풀기

이제 XOR 문제를 풀어보도록 하겠습니다. 일반적인 Logistic Regression으로 먼저 풀어볼텐데요, 먼저 XOR 데이터를 생성합니다. XOR은 각 숫자가 같을 때는 0, 다를 때는 1을 출력하는 논리구조입니다.

In [5]:
import numpy as np

x_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
y_data = np.array([[0], [1], [1], [0]], dtype=np.float32)

print(x_data)

[[0. 0.]
 [0. 1.]
 [1. 0.]
 [1. 1.]]


이제 Hypothesis를 정해보도록 하죠. Weight는 2개의 데이터가 들어가고 하나의 데이터로 나오기 때문에 2 x 1 의 행렬입니다. bias는 출력되는 데이터와 같으므로 1차원 구조입니다.

In [7]:
import tensorflow as tf

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)
W = tf.Variable(tf.random_normal([2, 1]), name="weight")
b = tf.Variable(tf.random_normal([1]), name="bias")

hypothesis = tf.sigmoid(tf.matmul(X, W) + b)

cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(10001):
        sess.run(train, feed_dict={X: x_data, Y: y_data})
        if step % 100 == 0:
            print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}), sess.run(W))
            
    ## x_data, y_data 학습데이터 그대로 테스트
    h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict={X: x_data, Y: y_data})
    print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)

0 1.2679496 [[-0.40582794]
 [-1.0018556 ]]
100 0.696044 [[ 0.25111973]
 [-0.09925942]]
200 0.69397414 [[ 0.1578951 ]
 [-0.02948391]]
300 0.69343066 [[ 0.09363437]
 [-0.00649389]]
400 0.6932497 [[0.05616169]
 [0.0026675 ]]
500 0.6931863 [[0.03414351]
 [0.00556552]]
600 0.6931629 [[0.02103469]
 [0.00576785]]
700 0.6931537 [[0.01312332]
 [0.00496756]]
800 0.69315004 [[0.00828382]
 [0.00392691]]
900 0.6931484 [[0.00528466]
 [0.00295716]]
1000 0.6931478 [[0.00340315]
 [0.00215977]]
1100 0.6931474 [[0.0022095 ]
 [0.00154528]]
1200 0.6931473 [[0.00144459]
 [0.00108978]]
1300 0.69314724 [[0.00095007]
 [0.00076053]]
1400 0.6931472 [[0.00062788]
 [0.00052664]]
1500 0.6931472 [[0.00041672]
 [0.00036264]]
1600 0.6931472 [[0.00027747]
 [0.00024858]]
1700 0.6931472 [[0.00018523]
 [0.0001698 ]]
1800 0.6931472 [[0.00012393]
 [0.00011569]]
1900 0.6931472 [[8.305711e-05]
 [7.865582e-05]]
2000 0.6931472 [[5.574478e-05]
 [5.339389e-05]]
2100 0.6931472 [[3.7455095e-05]
 [3.6200930e-05]]
2200 0.6931472 [[2.

이렇게 일반적인 Logistic Regression으로 학습을 시키고 테스트를 해보면 학습 데이터를 그대로 줬음에도 불구하고 정확도가 50%밖에 되지 않습니다. 즉, XOR 문제를 잘 풀지 못하는거죠. 이제 hidden layer를 넣어서 학습시켜보도록 하겠습니다.

[ 0 0 ] ======> [ 2 x 2 행렬 (weight) ] + [ bias ] = [ 1 x 2 행렬 ] =======> [ 2 x 1 행렬 (weight) ] + [ bias ] = 하나의 값

의 Multi layer 형태로 학습을 시켜보겠습니다.

In [8]:
import tensorflow as tf

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)
# W = tf.Variable(tf.random_normal([2, 1]), name="weight")
# b = tf.Variable(tf.random_normal([1]), name="bias")
# hypothesis = tf.sigmoid(tf.matmul(X, W) + b)

W1 = tf.Variable(tf.random_normal([2, 2]), name="wieght1")
b1 = tf.Variable(tf.random_normal([2]), name="bias1")
layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)

W2 = tf.Variable(tf.random_normal([2, 1]), name="wieght2")
b2 = tf.Variable(tf.random_normal([1]), name="bias2")
hypothesis = tf.sigmoid(tf.matmul(layer1, W2) + b2)

cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(10001):
        sess.run(train, feed_dict={X: x_data, Y: y_data})
        if step % 100 == 0:
            print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}), sess.run(W))
            
    ## x_data, y_data 학습데이터 그대로 테스트
    h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict={X: x_data, Y: y_data})
    print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)

0 0.9915812 [[-1.5572703 ]
 [-0.56141967]]
100 0.703548 [[-1.5572703 ]
 [-0.56141967]]
200 0.6974315 [[-1.5572703 ]
 [-0.56141967]]
300 0.6929913 [[-1.5572703 ]
 [-0.56141967]]
400 0.68941087 [[-1.5572703 ]
 [-0.56141967]]
500 0.68626773 [[-1.5572703 ]
 [-0.56141967]]
600 0.6832745 [[-1.5572703 ]
 [-0.56141967]]
700 0.6802027 [[-1.5572703 ]
 [-0.56141967]]
800 0.67685133 [[-1.5572703 ]
 [-0.56141967]]
900 0.67302775 [[-1.5572703 ]
 [-0.56141967]]
1000 0.6685319 [[-1.5572703 ]
 [-0.56141967]]
1100 0.66314125 [[-1.5572703 ]
 [-0.56141967]]
1200 0.6565964 [[-1.5572703 ]
 [-0.56141967]]
1300 0.64858156 [[-1.5572703 ]
 [-0.56141967]]
1400 0.6386995 [[-1.5572703 ]
 [-0.56141967]]
1500 0.62643564 [[-1.5572703 ]
 [-0.56141967]]
1600 0.61111414 [[-1.5572703 ]
 [-0.56141967]]
1700 0.5918702 [[-1.5572703 ]
 [-0.56141967]]
1800 0.5677097 [[-1.5572703 ]
 [-0.56141967]]
1900 0.5377759 [[-1.5572703 ]
 [-0.56141967]]
2000 0.50185573 [[-1.5572703 ]
 [-0.56141967]]
2100 0.46087083 [[-1.5572703 ]
 [-0.56

이렇게 Multi layer를 사용하면 XOR문제를 풀 수 있습니다. Hidden layer에서 node(perceptron)을 더 많이 사용할 수도 있습니다. (node를 많이 사용할 수록 "wide하다"라고 합니다.) 더 wide하게 사용하면 optimize를 더 효과적으로 할 수 있습니다. 게다가 layer를 더 많이 사용해도 (layer를 많이 사용할수록 "deep하다"라고 합니다.) 효과적인 학습을 할 수 있습니다.